# Парсеры для скрапинга информации с сайта LinkedIn

In [1]:
# вот эта штука не оставляет следов по которым всякие антиботы
# могут нас поймать
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from urllib.parse import unquote
from bs4 import BeautifulSoup
import random
from tqdm.notebook import tqdm
import re
import time
import json

In [2]:
def open_linkedin(filename='path_to_the_file.txt'):
    """
    Opens undetected chromedriver
    and log in to linkedIn

    Parameters
    ----------
    filename :
         (Default value = 'path_to_the_file.txt')
         A path to the txt file where
         login and password are stored

    Returns
    -------
        undetected webdriver
    """
    with open(filename, 'r') as handler:
        lines = handler.readlines()
        MAIL, PASS = lines[0].strip(), lines[1]
    
    
    driver = uc.Chrome()
    driver.get("https://linkedin.com/uas/login")
    time.sleep(3)
    login_field = driver.find_element(By.ID,"username")
    login_field.send_keys(MAIL)

    pass_field = driver.find_element(By.ID,"password")
    pass_field.send_keys(PASS)
    pass_field.submit()
    time.sleep(4)
    
    return driver


In [10]:
def profiles_search_results(driver, 
                            query:str, 
                            pages_num:int = 5)->list:
    """
    Search profilles according to the
    search query

    Parameters
    ----------
    driver :
        undetected webdriver
    query : str
        Query search
    pages_num : int
         (Default value = 5)
        Number of pages to scrape
    Returns
    -------

    """
    
    #сюда складываю ссылки на профили
    profiles_links = []
    for page in range(1, pages_num+1):
        # нажатие кнопки next  у меня не работает, поэтому просто меняю страницу в самой url
        # локация Россия
        driver.get(f'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22101728296%22%5D&keywords={query}' +
                   f'&origin=FACETED_SEARCH&page={page}&sid=wMe')
        soup = BeautifulSoup(driver.page_source, 'lxml')
        # тут просто собираю ссылки
        for person in soup.find_all('span', attrs={'class': 'entity-result__title-text'}):
            link = person.find('a', attrs={'class': 'app-aware-link'})['href']
            profiles_links.append(link)
        # прокрутка до конца страницы для имитации реального пользователя
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(1,5))

    return profiles_links

In [ ]:
def get_company_emploeeys(driver, 
                          companies:list, 
                          n_scrolls:int = 1)->list:
    """
    Search profiles according to the
    companies

    Parameters
    ----------
    driver :
        undetected webdriver
    companies : list
        list of companies to search for
    n_scrolls : int
         (Default value = 1)
        Number of pages to scrape
    Returns
    -------
    companies_employeers: dict
        dict of companies and their
        employee's links
    """
    companies_employeers = {}
    for company in companies: 
        employees_links = []
        driver.get(f'https://www.linkedin.com/company/{company}/people/') 
        
        time.sleep(15) 
        # scroll + timeout 
        for _ in range(n_scrolls):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(5,10)) 
        soup = BeautifulSoup(driver.page_source, 'lxml')
        # org-people-profile-card__profile-info
        
        #сбор ссылок
        for index, person in enumerate(soup.find_all('div', attrs={'class': 'org-people-profile-card__profile-info'})):
            #print(index)
            try:
                image = person.find('div', attrs={'class':'artdeco-entity-lockup__image'})
                link = image.find('a', attrs={'class': 'app-aware-link'})['href']
                employees_links.append(link)
            except:
                continue
        companies_employeers[company] = employees_links
        time.sleep(random.uniform(1,5))  
          
    return companies_employeers
    

In [4]:
def get_company_and_experience(driver)->tuple:
    """
    Scrapes the experience page

    Parameters
    ----------
    driver :
        undetected webdriver

    Returns
    -------
    tuple of strings:
        company name
        years of total experience
        employment type
    """
    
    
    #переходим на страницу опыта
    experience_page = driver.current_url + 'details/experience/'
    driver.get(experience_page)
    time.sleep(random.uniform(1,5))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(driver.page_source,'lxml')
    total = 0
    #вот тут будут строчки из каждой позиции в опыте
    for i in soup.find_all('span', {'class':'t-14 t-normal t-black--light'}):
        # собираю тексты
        t = i.find('span', {'class': 'visually-hidden'}).text
        #не всегда есть год, месяц, иногда вообще пусто
        try:
            total += int(re.search(r'\d+ mo', t)[0].split()[0])
        except:
            pass
        try:
            total += int(re.search(r'\d+ yr', t)[0].split()[0]) * 12
        except:
            pass
    try:
        curr_job = (soup.find_all('li', {'class':'pvs-list__paged-list-item'})[0]
                         .find('span', 't-14 t-normal')
                         .find('span', 'visually-hidden')).text.strip()
        curr_company = curr_job.split(' · ')[0]
    except: 
        curr_job = None
        curr_company = None
    try:
        employment_type = curr_job.split(' · ')[1]
    except:
        employment_type = 'unknown'
        
    years = round(total / 12, 2)
    driver.back()
    
    return curr_company, years, employment_type

In [11]:
def get_skills(driver)->list:
    """
    Scrapes the skills page
    to get the skills list

    Parameters
    ----------
    driver :
        undetected webdriver

    Returns
    -------
    skills:
        list of profiles's skills 
    """
    
    curr_link = driver.current_url
    
    driver.get(curr_link + 'details/skills/')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source,'lxml')
    
    skills = []
    raw_skills = soup.find('main', 'scaffold-layout__main').find_all('div', 'pvs-entity')
    
    for skill in raw_skills:
        skills.append(skill.find('span', 'visually-hidden').text)
        
    driver.back()
    
    return list(set(skills))


In [12]:
def get_profile_info(driver, 
                     link:str)->tuple:
    """
    Scrapes the main profile page

    Parameters
    ----------
    driver :
        undetected chromedriver
        
    link : str
        Link for the profile's page

    Returns
    -------
    tuple of str and dict:
        u_id - unique person's id
        person - dictionary with all 
        scraped info
    """
    
    
    driver.get(link)
    time.sleep(random.uniform(2,5))
    soup = BeautifulSoup(driver.page_source,'lxml')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    person = {}
    u_id = unquote(link.split('in/')[1].split('?')[0])
    person['name'] = soup.find('h1', {'class': 'text-heading-xlarge'}).text
    person['city'] = soup.find('span',"text-body-small inline t-black--light break-words").text.strip().split(',')[0]
    try:
        person['connections'] = re.search('\d+. connections|\d+ connections', 
                                          soup.find('div', 'ph5')
                                .text)[0].split()[0]
    except:
        person['connections'] = '0'
    role = soup.find('div', {'class': 'text-body-medium break-words'}).text.strip()
    person['role'] = (role.split(' at ')[0]
                      .split(' - ')[0]
                      .split(' – ')[0]
                      .split(' as ')[0])
    
    person_info = get_company_and_experience(driver)
    person['company'] = person_info[0]
    person['experience'] = person_info[1]
    person['employment_type'] = person_info[2]
    
    person['skills'] = get_skills(driver)

    return u_id, person


In [7]:
def scrape_posts_reactions(driver, 
                           link:str, 
                           num_scrolls=5)->list:
    """
    Scrapes the posts or reactions
    page.

    Parameters
    ----------
    driver :
        undetected chromedriver
    link : str
        profile's link
    num_scrolls :
         (Default value = 5)
        number of max scrolls to scroll
        on the posts/reactions pages
    Returns
    -------
    posts_texts: list
        A list of texts
    """
    
    driver.get(link)
    time.sleep(random.uniform(3,5))
    last_height = driver.execute_script("return document.body.scrollHeight")
    #прокрутка до конца страницы
    for i in range(num_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(1,5))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    # собираю тексты
    soup = BeautifulSoup(driver.page_source,'lxml')
    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')
    posts_texts = []
    for post in posts:
        try:
            post_text = (post.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})
                         .find('span', {'dir': 'ltr'}).text)
            posts_texts.append(post_text)
        except:
            pass

    return posts_texts 

In [8]:
def get_posts_reactions(driver, 
                        link:str)->tuple:
    """
    Goes to posts and reactions page and
    scrapes them successively

    Parameters
    ----------
    driver :
        undetected chromedriver
    link : str
        profile's link

    Returns
    -------
    tuple of lists:
        posts and reactions
    """
    
    #тут есть два раздела: posts и reactions
    time.sleep(random.uniform(1,5))
    
    posts_url = driver.current_url + 'recent-activity/all/'
    reactions_url = driver.current_url + 'recent-activity/reactions/'
    # тексты для каждого
    posts = scrape_posts_reactions(driver, posts_url)
    time.sleep(random.uniform(1,5))
    react_posts = scrape_posts_reactions(driver, reactions_url)
        
    return posts, react_posts

In [ ]:
def parse_profiles(search_query:str, 
                   filename:str,
                   num_of_profiles=85)-> tuple:
    """
    Main function to parse the profiles

    Parameters
    ----------
    search_query : str
        input search query
    filename : str
        file where login and password
        are stored
    num_of_profiles :
         (Default value = 85)
         number of profiles to scrape

    Returns
    -------
    tuple of dictionaries
    """

    # соединяю запрос в одну строку заменяя пробелы
    search_query = '%20'.join(search_query.split())
    #открываю линкдин
    driver = open_linkedin(filename)
    #собираю ссылки со страницы резцльтатов поиска
    links = profiles_search_results(driver, search_query, pages_num=30)
    links = [link for link in links if 'in/' in link]
    # словарь профилей с информацией о них
    profiles = {}
    # словари с постами пользователей
    # и с постами на которые реагировал пользователь
    u_posts, u_reactions = {}, {}

    for count, link in tqdm(enumerate(links[:])):
        u_id, profile = get_profile_info(driver, link)
        profiles[u_id] = profile
        posts, reactions = get_posts_reactions(driver, link)
        u_posts[u_id] = posts
        u_reactions[u_id] = reactions
        if count == num_of_profiles:
            break
    
    return profiles, u_posts, u_reactions

In [9]:
def scroll_reactions_window(driver,
                            max_scrolls:int = 40):
    """
    Scroll the reactions window.

    Parameters
    ----------
    driver :
        
    max_scrolls : int
         (Default value = 40)

    Returns
    -------
        None
    """
    
    
    js_code = "return document.getElementsByClassName('artdeco-modal__content')[0].scrollHeight"
    last_height = driver.execute_script(js_code)
    count = 0
    while count < max_scrolls:
        time.sleep(random.uniform(1,3))
        scroll_script = 'arguments[0].scrollTop = arguments[0].scrollHeight'
        driver.execute_script(scroll_script, 
                              driver.find_element(By.CLASS_NAME,'artdeco-modal__content'))
        time.sleep(random.uniform(2,4))
        new_height = driver.execute_script(js_code)
        if new_height == last_height:
            break
        last_height = new_height
        count += 1

In [ ]:
def parse_posts_by_query(search_query:str,
                         filename:str,
                         max_posts:int = 150)->dict:
    """
    

    Parameters
    ----------
    search_query : str
        search query text
    filename : str
        filename with the login info
    max_posts : int
         (Default value = 150)->dictdriver = open_linkedin(filename)
        max posts to scrape
    Returns
    -------
    result_dict: dict
        A dictionary with all results
    """

    driver = open_linkedin(filename)
    
    # соединяю его в одну строку заменяя пробелы
    search_query = '%20'.join(search_query.split())

    #пустой словарь куда будем скидывать результаты
    result_dict = {}

    # переходим на страницу с нужным поисковым запросом
    driver.get(f'https://www.linkedin.com/search/results/content/?keywords={search_query}&origin=SWITCH_SEARCH_VERTICAL')
    time.sleep(2)

    #создаем 2 списка, в каждом из них элементы при нажатии на которые открываются лайки
    likes_list_1 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__reactions-count')
    likes_list_2 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__social-proof-text')

    #каунтер обозначает номер поста, 2 следующих указателя определяют номера элементов реакций
    count = 0
    reactions_button_counter_1 = 0
    reactions_button_counter_2 = 0
    # переменная soup отвечает за содержимое всей страницы
    soup = BeautifulSoup(driver.page_source,'lxml')
    
    while len(result_dict)<=max_posts:
        time.sleep(random.uniform(2,4))
        # переменная post_card отвечает за содержимое карточки поста
        post_card = soup.find_all('div','pt1 mb2 artdeco-card')[count]
        try:
            #пытаемся вытащить текст поста
            text_relative_feed = post_card.find('div','update-components-text relative feed-shared-update-v2__commentary')
            post_text = text_relative_feed.find('span',{'dir':'ltr'}).text
            result_dict[count] = [post_text,{}]
        except:
            #если текста нет и в посте только картинка, то так и пишем
            result_dict[count] = ['Just image in post',{}]
            print('Just image in post')

        #проверяем условие по которому делается клик на соответствующую кнопку с лайками
        if 'social-details-social-counts__reactions-count' in str(post_card):
            likes_list_1[reactions_button_counter_1].click()
            reactions_button_counter_1 += 1
        elif 'social-details-social-counts__social-proof-text' in str(post_card):
            likes_list_2[reactions_button_counter_2].click()
            reactions_button_counter_2 += 1
        else:
            count += 1
            print('error')
            continue

        scroll_reactions_window(driver)

        time.sleep(random.uniform(2,6))
        #кнопка с лайками нажата, парсим всю инфу со страницы в новый суп
        reactions_window_soup = BeautifulSoup(driver.page_source,'lxml')
        #все лайкнувшие юзеры будут тут, но нужно скролить до конца
        reactions_window_soup = reactions_window_soup.find('ul','artdeco-list artdeco-list--offset-1')
        usernames = []
        user_links = []
        job_titles = []
        for i in reactions_window_soup.find_all('a', 'link-without-hover-state ember-view'):
            try:
                name = i.find('span',{'dir':'ltr'}).text
            except:
                name = 'Not defined'
            usernames.append(name)
            user_link = i['href']
            user_links.append(user_link)
            job_title = i.find('div','artdeco-entity-lockup__caption ember-view').text.strip()
            job_titles.append(job_title)

        # упаковываем результаты в словарь
        for i,v in enumerate(user_links):
            result_dict[count][1][v] = [usernames[i],job_titles[i]]
        #закрываем страницу с лайками и добавляем к каунтеру 1
        driver.find_element(By.CLASS_NAME, 'artdeco-button__icon').click()
        count += 1

        time.sleep(random.uniform(2,7))
        #создаем 2 списка, в каждом из них элементы при нажатии на которые открываются лайки
        likes_list_1 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__reactions-count')
        likes_list_2 = driver.find_elements(By.CLASS_NAME, 'social-details-social-counts__social-proof-container')
        time.sleep(random.uniform(5,10))
        soup = BeautifulSoup(driver.page_source,'lxml')
        
        return result_dict


# Сбор информации

Профили и информация по ним на примере поискового запроса "senior developer"

In [ ]:
query = 'senior developer'

profiles, u_posts, u_reactions = parse_profiles(query,
                                                'my_login.txt')

Посты и информация о реакциях на примере запроса "менторство в it"

In [ ]:
query = 'менторство в it'

posts_results = parse_posts_by_query(query,
                                     'my_login.txt')